# SQL Window Functions - Calculating Aggregates

### Introduction 

In this lesson, we'll learn about SQL window functions.  And in doing so, we'll see how we can use window functions to perform aggregations within a specific grouping of rows.  We'll also see how, unlike aggregate functions with group by, these functions do not reduce the number of rows returned, but rather just add an extra column with the aggregate.  Ok, let's get started.

### Loading our Data

We can begin by loading our data from the [favorita kaggle competition](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data).

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-eng-10-21/window-functions/main/favorita_transactions.csv"
df = pd.read_csv(url)
df[:2]

,id,date,store_nbr,transactions
0,0,2013-01-01,25,770
1,1,2013-01-02,1,2111


Above, we just loaded sales data from different favorita grocery chains on different days. 

Ok, now let's load this data into our database.

In [2]:
import sqlite3
conn = sqlite3.connect('grocery.db')

In [3]:
df.to_sql('store_transactions', conn, index = False, if_exists = 'replace')

In [4]:
pd.read_sql('SELECT * FROM store_transactions LIMIT 1', conn)

,id,date,store_nbr,transactions
0,0,2013-01-01,25,770


### Introducing Window Functions

Now window functions are often compared to group by functions.  And this is because, like group by functions, window functions allow us to perform aggregate calculations.  

Let's review with this.

In [5]:
query = '''SELECT AVG(transactions)
            as avg_transactions 
            FROM store_transactions'''
pd.read_sql(query, conn)

,avg_transactions
0,1694.602158


So above, we just calculated the average amount of transactions across our entire dataset.

Now let's see a similar query, but this time we'll use  a window function.

In [6]:
query = '''SELECT date, store_nbr, transactions, 
AVG(transactions) OVER ()
            as avg_transactions 
            FROM store_transactions
            LIMIT 5'''
pd.read_sql(query, conn)

,date,store_nbr,transactions,avg_transactions
0,2013-01-01,25,770,1694.602158
1,2013-01-02,1,2111,1694.602158
2,2013-01-02,2,2358,1694.602158
3,2013-01-02,3,3487,1694.602158
4,2013-01-02,4,1922,1694.602158


So we with our window function, we add the `OVER` keyword.  And we can see that as a result, SQL does calculate the average transactions, but it does not reduce the number of rows.

This is a useful feature if we want to, for example, calculate how much a store's transactions deviates from the average.  We can do so with the following.

In [37]:
query = '''SELECT date, store_nbr, 
transactions, transactions - AVG(transactions) OVER ()
            as diff_from_avg
            FROM store_transactions LIMIT 5'''
pd.read_sql(query, conn)

,date,store_nbr,transactions,diff_from_avg
0,2013-01-01,25,770,-924.602158
1,2013-01-02,1,2111,416.397842
2,2013-01-02,2,2358,663.397842
3,2013-01-02,3,3487,1792.397842
4,2013-01-02,4,1922,227.397842


### Understanding the Window Function

Now let's take another look at our original query using window functions.

```sql
SELECT date, store_nbr, transactions, 
AVG(transactions) OVER ()
            as avg_transactions 
            FROM store_transactions
```

The window component is the `AVG(transactions) OVER ()` and the inside the parentheses after Over, we define the "window".  The window just means the group of rows to consider.

In the above query, we do not specify a subset of rows, so SQL calculates the average across *all* of the rows.

However, if we change the window to be the `store_nbr` -- the store number -- then this time we will calculate the average number of transactions per each store.  Let's see this. 

In [47]:
query = '''SELECT date, store_nbr, 
transactions, AVG(transactions) OVER (partition by store_nbr)
            as avg_by_store
            FROM store_transactions
            WHERE store_nbr = 1 
            LIMIT 2'''
pd.read_sql(query, conn)

,date,store_nbr,transactions,avg_by_store
0,2013-01-02,1,2111,1523.844272
1,2013-01-03,1,1833,1523.844272


In [48]:
query = '''SELECT date, store_nbr, 
transactions, AVG(transactions) OVER (partition by store_nbr)
            as avg_by_store
            FROM store_transactions
            WHERE store_nbr = 2 
            LIMIT 2'''
pd.read_sql(query, conn)

,date,store_nbr,transactions,avg_by_store
0,2013-01-02,2,2358,1920.036374
1,2013-01-03,2,2033,1920.036374


So this time, we can see that we get different averages based on the store.  And to achieve this, we added the phrase `partition by` to our window function. 

```sql
AVG(transactions) OVER (partition by store_nbr)
```

So as we can see in the last set of parentheses, we specify how we want to partition our rows, here by store number.

### Summary

In this lesson, we saw how window functions allow us to perform calculations within a specified window.  Unlike our aggregate functions, window functions do not reduce the number of rows that are returned.  This can make them useful for comparing a specific row against the calculated value in that window -- like deviation from the average.

We saw that we create a window function with the `OVER` keyword, and that in the parentheses after the OVER, the window is specified.  If we leave it blank, the window is all of the queried rows.

```sql
SELECT date, store_nbr, transactions, 
AVG(transactions) OVER ()
            as avg_transactions 
            FROM store_transactions
```

Or we can partition our dataset into different windows by a specified criteria.

```sql
SELECT date, store_nbr, 
transactions, AVG(transactions) OVER (partition by store_nbr)
            as avg_by_store
FROM store_transaction
```

### Resources

[Snowflake Window Functions](https://docs.snowflake.com/en/user-guide/functions-window-using.html)

[Data School Window Functions](https://dataschool.com/how-to-teach-people-sql/how-window-functions-work/)

[Kaggle Analytic Window Functions](https://www.kaggle.com/alexisbcook/analytic-functions)

[StrataScratch Window Functions](https://www.stratascratch.com/blog/types-of-window-functions-in-sql-and-questions-asked-by-airbnb-netflix-twitter-and-uber/)

[Instacard Data](https://www.kaggle.com/c/instacart-market-basket-analysis/data)

[Chartio Window Functions](https://chartio.com/resources/tutorials/using-window-functions/)